In [40]:
import pandas as pd
import pickle

In [33]:
path = "../../datathon/datathon"

outfit_data = pd.read_csv(path + "/dataset/outfit_data.csv")
product_data = pd.read_csv(path + "/dataset/product_data.csv")

In [35]:
outfit_data['cod_modelo_color'] = outfit_data['cod_modelo_color'].str.slice(stop=-3)
outfit_data = outfit_data.drop_duplicates()
merged_df = pd.merge(outfit_data, outfit_data, how='left', on='cod_outfit')
merged_df = merged_df[merged_df['cod_modelo_color_x'] != merged_df['cod_modelo_color_y']]
merged_df = merged_df.groupby('cod_modelo_color_x').agg({'cod_modelo_color_y': lambda x: list(x)}).reset_index()
merged_df

,cod_modelo_color_x,cod_modelo_color_y
0,37000,"[47087, 57050, 57081, 57003, 57010, 57083, 570..."
1,37010,"[47094, 57020, 57001, 47034, 57010, 57040, 470..."
2,37027,"[47005, 47095, 57010, 47050]"
3,37040,"[57024, 57046, 57005, 57006, 57025, 57034]"
4,37050,"[57050, 47015, 57091, 57032]"
...,...,...
453,67091,"[57095, 67020, 67000, 57029]"
454,67106,"[57025, 57095, 57037, 57088]"
455,87040,"[57093, 57000, 57001, 57010]"
456,87062,"[57040, 57071, 57063, 57073]"


In [39]:
compatibilities = dict(zip(merged_df.cod_modelo_color_x, merged_df.cod_modelo_color_y))

with open(path + '/dataset/compatibilities.pickle', 'wb') as handle:
    pickle.dump(compatibilities, handle, protocol=pickle.HIGHEST_PROTOCOL)